#### Introduction
Aim of this kernel is to create multilinear regression model and create REST API for this model.This model has been deployed in Heroku please find the linkat the end of this kernel.


#### What is REST API

Representational state transfer (REST) is a software architectural style that defines a set of constraints to be used for creating Web services.It means when a RESTful API is called, the server will transfer to the client a representation of the state of the requested resource.There must be a clear separation between client and server such that clients are not concerned with data storage and servers are not concerned with the User Interface.<br>
Stateless: State information is not stored on the server.  A client  request must contain all information like session to service the request.REST APIs can exchange data in either JSON or XML format, although many REST APIs send data as JSON.<br>
In simple words, an API is a (hypothetical) contract between 2 softwares saying if the user software provides input in a pre-defined format, the later with extend its functionality and provide the outcome to the user software.


#### REST API for Machine Learning Model

Once you have trained model ready in your local computer, needs to be deployed in order to be used by the clients or end users.By building a REST API for my model, it is easy to code separate from other developers and prevents  from directly blocking teammates who are not involved with the machine learning aspect of the project. Another advantage is that the model can be used by multiple developers working on different platforms, such as web or mobile.

#### FLASK

Flask is a web framework. This means flask provides you with tools, libraries and technologies that allow you to build a web application. This web application can be some web pages, a blog, a wiki or go as big as a web-based calendar application or a commercial website.A framework is a code storage that should help developers achieve the required result by making work easier, scalable, efficient and maintainable web applications by providing reusable code or extensions for common operations.<br>
Flask and connect it to HTML. Anytime any user sends information on the net or goes to the search bar, the HTML connects the user. The flask framework looks for HTML files (templates) in a folder called Templates. Before sending the template over, Python code is executed which injects variables, code etc.

#### Create Model

This dataset has data collected from New York, California and Florida about 50 business Startups "17 in each state". The variables used in the dataset are Profit, R&D spending, Administration Spending, and Marketing Spending.

In [ ]:
# Import Libraries
import pandas as pd
import pickle
from category_encoders import *
import matplotlib.pyplot as plt
# %matplotlib inline
import seaborn as sns

In [ ]:
train_df=pd.read_csv("../input/50-startups/50_Startups.csv")

In [ ]:
train_df.head(4)


In [ ]:
enc = OrdinalEncoder().fit(train_df)

In [ ]:
df_train_encoded = enc.transform(train_df)

df_train_encoded.head(5)

In [ ]:
corremat = df_train_encoded.corr()
top_corr_features = corremat.index[(corremat['Profit'])> .1]
plt.figure(figsize=(10,10))
g= sns.heatmap(train_df[top_corr_features].corr(),annot=True,cmap='viridis',linewidths=.5)

In [ ]:
X=df_train_encoded.iloc[:, :-1].values

In [ ]:
y = df_train_encoded.iloc[:, 4].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train,y_train)


In [ ]:
y_pred=regressor.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score


In [ ]:
r2_score(y_test,y_pred)

Comparing the actual with predicted one

In [ ]:
df =pd.DataFrame(data=y_test,columns=['y_test'])
df['y_pred'] = y_pred
df

In [ ]:
regressor.predict([[165349.20,	136897.80,	471784.10,1]])

Now our model is ready we will create .pkl file for the model

In [ ]:
import pickle
pickle.dump(regressor,open('startup_prediction.pkl','wb'))


#### HTML Page

 Now we need to create front end using HTML for the user to input the values. There are four fields (R&D spend,Admin expences,Marketing expenses,State)which need to be filled by the user.

In [ ]:
"""
<!DOCTYPE html>
<html >
<!--From https://codepen.io/frytyler/pen/EGdtg-->
<head>
  <meta charset="UTF-8">
  <title>ML API</title>
  <link href='https://fonts.googleapis.com/css?family=Pacifico' rel='stylesheet' type='text/css'>
<link href='https://fonts.googleapis.com/css?family=Arimo' rel='stylesheet' type='text/css'>
<link href='https://fonts.googleapis.com/css?family=Hind:300' rel='stylesheet' type='text/css'>
<link href='https://fonts.googleapis.com/css?family=Open+Sans+Condensed:300' rel='stylesheet' type='text/css'>
  
</head>

<style>
body {background-color: Lavender ;}
h1   {color: DarkBlue;}
h3   {color: FireBrick   ;}
p    {color: red;}
</style>
<body>
 <div class="login">
	<h1>Predict Profit for Startup</h1>

     <!-- Main Input For Receiving Query to our ML -->
    <form action="{{ url_for('predict')}}"method="post">
	
	<h3>Enter 1 to select the State  </h3>
    	
		
	
		<br>
		<br>
		<input type="text" name="R&D Spent" placeholder="R&D Expence" required="required" />
        <input type="text" name="Administration Expence" placeholder="Administration Expence" required="required" />
		<input type="text" name="Marketing Expence" placeholder="Marketing Expence" required="required" />	
		
		
		<label for="relation">State</label>
<select id="relation" name="relation">
	<option value=1>New York</option>
	<option value=2>California</option>
	<option value=3>Florida</option>
	</select>
		

		
        <button type="submit" required="required" class="btn btn-primary btn-block btn-large">Predict </button>
    </form>

   <br>
   <br>
   {{ prediction_text }}

 </div>


</body>
</html>
"""

> 

#### REST API with FLASK
Below is the code for creating REST API ,we will check each block of code saparately

In [ ]:

"""

import numpy as np
from flask import Flask,request,jsonify,render_template
import pickle

app = Flask(__name__)
salary_deploy = pickle.load(open('startup_prediction.pkl','rb'))

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict',methods = ['POST'])

def predict():
    int_features = [float(x) for x in request.form.values()]
    final_features = [np.array(int_features)]
    prediction = salary_deploy.predict(final_features)
    
    output = round(prediction[0],2)
    
    return render_template('index.html',prediction_text ='Profit for the startup should be $ {}'.format(output))


@app.route('/predict_api',methods=['POST'])
def predict_api():

    data = request.get_json(force=True)
    prediction = salary_deploy.predict([np.array(list(data.values()))])

    output = prediction[0]
    return jsonify(output)


if __name__ == '__main__':
    app.run(debug=True)
    """

**import numpy as np<br>
from flask import Flask,request,jsonify,render_template<br>
import pickle**<br>

##import - Importing three libraries numpy,flaskand pickle<br>
##render_template -redircting to home page(html form)

**app = Flask(__name__)<br>
salary_deploy = pickle.load(open('startup_prediction.pkl','rb'))**<br>

##Initializing the Flask app and opening the pickle file in read mode<br>
<h4>
@app.route('/')<br>
def home():<br>
    return render_template('index.html') </h4>
##Create a URL route node in our application,it will redirect to index.html page<br>

<h4>
@app.route('/predict',methods = ['POST'])
</h4> 
    ##defines the method type and url to be hit by client for this method<br>
  <h4>  
def predict():<br>
    int_features = [float(x) for x in request.form.values()]<br>
    final_features = [np.array(int_features)]<br>
    prediction = salary_deploy.predict(final_features) </h4><br>
##this  function is to rendering result on html GUI
##After giving the inputs, Since it is post method the value will be read it from request.form.values and saved in int_features.Then it will be converted in to array.finally will predict.<br>
   
  ** output = round(prediction[0],2)** <br>
##will get the values in output after rounding off
    
 <h4>   return render_template('index.html',prediction_text ='Profit for the startup should be $ {}'.format(output))  </h4> 
##return the render_template in 'index.html' and prediction_text place holder in html will replace with predicted value.
<h4>

@app.route('/predict_api',methods=['POST'])<br>
def predict_api():<br>  
    data = request.get_json(force=True)<br>
    prediction = salary_deploy.predict([np.array(list(data.values()))])<br>
    output = prediction[0]<br>
    return jsonify(output)</h4> <br>
    
##For direct API calls trough request


In [ ]:
#Hardcoded json input for the direct api call

"""
import requests

url = 'http://localhost:5000//predict_api'
r = requests.post(url,json={'R&D Spend':122, 'Administration':333, 'Marketing Spend':444,'State':1})

print(r.json())
"""


Once you have built your model and REST API and finished testing locally, you can deploy your API just as you would any Flask app to the many hosting services on the web. By deploying on the web, users everywhere can make requests to your URL to get predictions

#### File we created
startup_prediction.pkl — This contains code for the machine learning model to predict the profit.<br>
app.py — This contains Flask APIs that receives inputs through GUI or API calls, computes the predicted value based on our model and returns it.<br>
request.py — This uses requests module to call APIs defined in app.py and displays the returned value.<br>
index.html — This contains the HTML template  to allow user to enter  detail and displays the predicted profit and this file has to be in a folder called 'templates'.


Run app.py file using this command in terminal    $ python app.py

Open this URL('http://127.0.0.1:5000/') in web browser to get GUI in your local machine.

Here is the link after deployment on cloud

https://mlprojectdeployed.blogspot.com